# Neural Operator Training Demo: CDON Dataset

This notebook demonstrates end-to-end training of neural operator models (DeepONet, FNO, UNet) on the CDON dataset.

**Features:**
- Trains on **real CDON data**
- **Sequential training with all 3 loss functions** (Baseline, BSP, SA-BSP)
- **Multi-loss comparison plots** showing training metrics
- **Energy spectrum visualization** (E(k) vs wavenumber) to identify spectral bias
- **Spectral bias quantification** with metrics and comparison plots
- Compatible with Google Colab

**Models available:**
- `deeponet`: Branch-trunk architecture with SIREN activation (~235K params)
- `fno`: Fourier Neural Operator (~261K params)
- `unet`: Encoder-decoder with skip connections (~249K params)

## Cell 1: Setup & Imports (Colab-Ready)

In [ ]:
# Google Colab setup
import sys
import os
from pathlib import Path

# Ensure we're in /content
try:
    os.chdir('/content')
except:
    pass

# Clone repository if running in Colab
repo_path = Path('/content/local')
if not repo_path.exists():
    print("📥 Cloning repository...")
    !git clone https://github.com/maximbeekenkamp/local.git
    print("✅ Repository cloned")
else:
    print("✅ Repository exists")

# Change to repo directory
try:
    os.chdir('/content/local')
    print(f"✅ Changed to: {os.getcwd()}")
except:
    pass

# Install dependencies
print("\n📦 Installing dependencies...")
!pip install -r requirements.txt -q
print("✅ Dependencies installed")

# Standard imports
import torch
import matplotlib.pyplot as plt
import numpy as np
from torch.utils.data import DataLoader

# Project imports
from src.core.data_processing.cdon_dataset import CDONDataset
from src.core.data_processing.cdon_transforms import CDONNormalization
from src.core.models.model_factory import create_model
from src.core.training.simple_trainer import SimpleTrainer
from configs.training_config import TrainingConfig

print("\n✓ Imports successful")
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")

## Cell 2: Load Real CDON Data

In [ ]:
# Get project root
project_root = Path.cwd()
print(f"Project root: {project_root}")

# Data directory
DATA_DIR = project_root / 'CDONData'
print(f"Data directory: {DATA_DIR}")

# Create normalization object (required by CDONDataset)
stats_path = project_root / 'configs' / 'cdon_stats.json'
print(f"Loading stats from: {stats_path}")
normalizer = CDONNormalization(stats_path=str(stats_path))

# Create datasets
train_dataset = CDONDataset(
    data_dir=str(DATA_DIR),
    split='train',
    normalize=normalizer
)

val_dataset = CDONDataset(
    data_dir=str(DATA_DIR),
    split='test',
    normalize=normalizer
)

# Create dataloaders
BATCH_SIZE = 16

train_loader = DataLoader(
    train_dataset,
    batch_size=BATCH_SIZE,
    shuffle=True,
    num_workers=2,
    pin_memory=True
)

val_loader = DataLoader(
    val_dataset,
    batch_size=BATCH_SIZE,
    shuffle=False,
    num_workers=2,
    pin_memory=True
)

print(f"\n✓ Data loaded successfully")
print(f"  Train samples: {len(train_dataset)}")
print(f"  Val samples: {len(val_dataset)}")
print(f"  Batch size: {BATCH_SIZE}")

# Inspect a sample
sample_input, sample_target = train_dataset[0]
print(f"\nSample shapes:")
print(f"  Input: {sample_input.shape}")
print(f"  Target: {sample_target.shape}")

## Cell 3: Choose Model Architecture

**Change `MODEL_ARCH` to try different models:**
- `'deeponet'`: Branch-trunk architecture with SIREN activation
- `'fno'`: Fourier Neural Operator
- `'unet'`: U-Net encoder-decoder

In [ ]:
# Choose model architecture
MODEL_ARCH = 'deeponet'  # Options: 'deeponet', 'fno', 'unet'

model = create_model(MODEL_ARCH)
num_params = sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f"✓ Created {MODEL_ARCH.upper()} model")
print(f"  Parameters: {num_params:,}")

## Cell 4: Initialize Results Storage

We'll train with all 3 loss types sequentially and store results for comparison.

In [ ]:
# Import loss configurations
from configs.loss_config import BASELINE_CONFIG, BSP_CONFIG, SA_BSP_CONFIG
from src.core.evaluation.loss_factory import create_loss

# Loss configuration map
loss_config_map = {
    'baseline': BASELINE_CONFIG,
    'bsp': BSP_CONFIG,
    'sa-bsp': SA_BSP_CONFIG
}

# Storage dictionaries for results from all loss types
all_training_results = {}  # Key: f"{MODEL_ARCH}_{loss_type}"
all_trainers = {}
trained_models = {}

print("✓ Storage initialized for multi-loss training")
print("\nWill train with 3 loss types:")
print("  1. BASELINE:", BASELINE_CONFIG.description)
print("  2. BSP:", BSP_CONFIG.description)
print("  3. SA-BSP:", SA_BSP_CONFIG.description)

## Cell 5: Sequential Training with All Loss Types

Train the same model architecture with all 3 loss functions sequentially.

In [ ]:
# Train with all 3 loss types sequentially
loss_types_to_train = ['baseline', 'bsp', 'sa-bsp']

for LOSS_TYPE in loss_types_to_train:
    print(f"\n{'='*70}")
    print(f"Training {MODEL_ARCH.upper()} with {LOSS_TYPE.upper()} Loss")
    print(f"{'='*70}\n")
    
    # Select loss configuration
    selected_loss_config = loss_config_map[LOSS_TYPE]
    print(f"Loss config: {selected_loss_config.description}")
    
    # Create loss function
    criterion = create_loss(selected_loss_config)
    print(f"✓ Loss function created: {type(criterion).__name__}")
    
    # Create FRESH model for this loss type (important!)
    model_for_loss = create_model(MODEL_ARCH)
    num_params = sum(p.numel() for p in model_for_loss.parameters() if p.requires_grad)
    print(f"✓ Fresh model created ({num_params:,} parameters)")
    
    # Create training config
    config = TrainingConfig(
        num_epochs=50,
        learning_rate=1e-3,
        batch_size=BATCH_SIZE,
        weight_decay=1e-4,
        scheduler_type='cosine',
        cosine_eta_min=1e-6,
        eval_metrics=['field_error', 'spectrum_error'],
        eval_frequency=1,
        checkpoint_dir=f'checkpoints/{MODEL_ARCH}_{LOSS_TYPE}',
        save_best=False,
        save_latest=False,
        device='cuda' if torch.cuda.is_available() else 'cpu',
        num_workers=2,
        verbose=True
    )
    
    # Create trainer
    trainer = SimpleTrainer(
        model=model_for_loss,
        train_loader=train_loader,
        val_loader=val_loader,
        config=config,
        loss_config=selected_loss_config,
        experiment_name=f'{MODEL_ARCH}_{LOSS_TYPE}'
    )
    
    print(f"✓ Trainer initialized")
    print(f"  Device: {trainer.device}")
    print(f"  Optimizer: {type(trainer.optimizer).__name__}")
    
    # Check for weight optimizer (SA-BSP only)
    if LOSS_TYPE == 'sa-bsp':
        if trainer.weight_optimizer is not None:
            print(f"  Weight optimizer: ✓ Created for SA-BSP")
        else:
            print(f"  ⚠ WARNING: SA-BSP but no weight_optimizer!")
    
    print(f"\n🚀 Starting training...\n")
    
    # Train
    results = trainer.train()
    
    # Store results
    key = f"{MODEL_ARCH}_{LOSS_TYPE}"
    all_training_results[key] = results
    all_trainers[key] = trainer
    trained_models[key] = model_for_loss
    
    print(f"\n✅ {LOSS_TYPE.upper()} training complete!")
    print(f"   Best val loss: {results['best_val_loss']:.6f}")
    print(f"   Final val loss: {results['val_history'][-1]['loss']:.6f}")

print(f"\n{'='*70}")
print(f"ALL TRAINING COMPLETE!")
print(f"{'='*70}")
print(f"Trained {len(all_training_results)} models with different loss functions")

## Cell 6: Multi-Loss Training Comparison

Compare training metrics across all 3 loss functions.

In [ ]:
# Create multi-loss comparison plots
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

# Color scheme for loss types
colors = {'baseline': '#1f77b4', 'bsp': '#ff7f0e', 'sa-bsp': '#2ca02c'}
linestyles = {'baseline': '-', 'bsp': '--', 'sa-bsp': '-.'}
markers = {'baseline': 'o', 'bsp': 's', 'sa-bsp': '^'}

for loss_type in ['baseline', 'bsp', 'sa-bsp']:
    key = f"{MODEL_ARCH}_{loss_type}"
    results = all_training_results[key]
    
    # Extract metrics
    val_losses = [h['loss'] for h in results['val_history']]
    val_field_errors = [h['field_error'] for h in results['val_history']]
    val_spectrum_errors = [h['spectrum_error'] for h in results['val_history']]
    epochs = range(1, len(val_losses) + 1)
    
    # Plot on all 3 axes
    axes[0].plot(epochs, val_losses, label=loss_type.upper(), 
                color=colors[loss_type], linestyle=linestyles[loss_type],
                linewidth=2, alpha=0.9, marker=markers[loss_type], markersize=4, markevery=5)
    
    axes[1].plot(epochs, val_field_errors, label=loss_type.upper(),
                color=colors[loss_type], linestyle=linestyles[loss_type],
                linewidth=2, alpha=0.9, marker=markers[loss_type], markersize=4, markevery=5)
    
    axes[2].plot(epochs, val_spectrum_errors, label=loss_type.upper(),
                color=colors[loss_type], linestyle=linestyles[loss_type],
                linewidth=2, alpha=0.9, marker=markers[loss_type], markersize=4, markevery=5)

# Configure axes
axes[0].set_xlabel('Epoch', fontsize=12)
axes[0].set_ylabel('Validation Loss', fontsize=12)
axes[0].set_title('Validation Loss Comparison', fontsize=14, fontweight='bold')
axes[0].legend(fontsize=11, loc='best')
axes[0].grid(True, alpha=0.3)

axes[1].set_xlabel('Epoch', fontsize=12)
axes[1].set_ylabel('Field Error', fontsize=12)
axes[1].set_title('Field Error (Real Space)', fontsize=14, fontweight='bold')
axes[1].legend(fontsize=11, loc='best')
axes[1].grid(True, alpha=0.3)

axes[2].set_xlabel('Epoch', fontsize=12)
axes[2].set_ylabel('Spectrum Error', fontsize=12)
axes[2].set_title('Spectrum Error (Frequency Space)', fontsize=14, fontweight='bold')
axes[2].legend(fontsize=11, loc='best')
axes[2].grid(True, alpha=0.3)

plt.suptitle(f'{MODEL_ARCH.upper()}: Loss Function Comparison', 
             fontsize=16, fontweight='bold')
plt.tight_layout()
plt.show()

# Print final metrics table
print(f"\n{'='*70}")
print("Final Metrics Summary")
print(f"{'='*70}")
print(f"{'Loss Type':<15} {'Val Loss':<12} {'Field Error':<15} {'Spectrum Error':<15}")
print("-"*70)

for loss_type in ['baseline', 'bsp', 'sa-bsp']:
    key = f"{MODEL_ARCH}_{loss_type}"
    results = all_training_results[key]
    final_val = results['val_history'][-1]
    
    print(f"{loss_type.upper():<15} {final_val['loss']:<12.6f} "
          f"{final_val['field_error']:<15.6f} {final_val['spectrum_error']:<15.6f}")

## Cell 7: Spectral Bias Visualization (Energy Spectrum)

Visualize E(k) vs wavenumber to identify spectral bias in trained models.

In [ ]:
import torch.fft as fft

def compute_energy_spectrum(signal, n_bins=32):
    """
    Compute time-averaged energy spectrum E(k) vs wavenumber.
    
    Args:
        signal: [batch, channels, timesteps] tensor
        n_bins: Number of wavenumber bins
    
    Returns:
        wavenumbers: array of wavenumber bin centers
        energy: E(k) energy spectrum values
    """
    # FFT along time dimension
    fft_signal = fft.rfft(signal, dim=-1)  # [B, C, freq]
    
    # Compute power: |FFT|^2
    power = torch.abs(fft_signal) ** 2  # [B, C, freq]
    
    # Average over batch and channels
    power_avg = power.mean(dim=(0, 1))  # [freq]
    
    # Create wavenumbers (k = 1, 2, ..., N)
    n_freq = power_avg.shape[0]
    wavenumbers = torch.arange(1, n_freq + 1, dtype=torch.float32)
    
    # Bin the frequencies into n_bins
    bin_edges = torch.linspace(0, n_freq, n_bins + 1)
    binned_energy = []
    binned_k = []
    
    for i in range(n_bins):
        start_idx = int(bin_edges[i])
        end_idx = int(bin_edges[i + 1])
        
        if end_idx > start_idx:
            # Average energy in this bin
            bin_energy = power_avg[start_idx:end_idx].mean().item()
            # Bin center wavenumber
            bin_k = wavenumbers[start_idx:end_idx].mean().item()
            
            binned_energy.append(bin_energy)
            binned_k.append(bin_k)
    
    return np.array(binned_k), np.array(binned_energy)


# Get validation batch for energy spectrum analysis
print("Computing energy spectra for all trained models...")
val_input, val_target = next(iter(val_loader))

# Move to device for inference
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
val_input = val_input.to(device)
val_target = val_target.to(device)

# Compute ground truth spectrum
k_true, E_true = compute_energy_spectrum(val_target)

# Compute spectrum for each trained model
spectra = {}
spectra['True'] = (k_true, E_true)

for loss_type in ['baseline', 'bsp', 'sa-bsp']:
    key = f"{MODEL_ARCH}_{loss_type}"
    model_trained = trained_models[key]
    model_trained.eval()
    model_trained.to(device)
    
    with torch.no_grad():
        pred = model_trained(val_input)
    
    k_pred, E_pred = compute_energy_spectrum(pred)
    spectra[f"{MODEL_ARCH.upper()} + {loss_type.upper()}"] = (k_pred, E_pred)
    
    print(f"  ✓ {loss_type.upper()} spectrum computed")

# Plot energy spectrum (like reference image)
fig, ax = plt.subplots(figsize=(10, 8))

# Plot ground truth (black solid line with higher zorder to be on top)
ax.loglog(k_true, E_true, 'k-', linewidth=3, label='True', zorder=10)

# Plot model predictions
colors_plot = {'baseline': '#1f77b4', 'bsp': '#ff7f0e', 'sa-bsp': '#2ca02c'}

for loss_type in ['baseline', 'bsp', 'sa-bsp']:
    label_key = f"{MODEL_ARCH.upper()} + {loss_type.upper()}"
    k, E = spectra[label_key]
    
    # Plot line
    ax.loglog(k, E, color=colors_plot[loss_type], linewidth=2.5, 
             alpha=0.8, label=label_key, zorder=5)

# Configure plot
ax.set_xlabel('Wavenumber', fontsize=14, fontweight='bold')
ax.set_ylabel('E(k)', fontsize=14, fontweight='bold')
ax.set_title(f'Time-Averaged Energy Spectrum Comparison\n{MODEL_ARCH.upper()} Model', 
            fontsize=16, fontweight='bold')
ax.legend(fontsize=12, loc='best', framealpha=0.9)
ax.grid(True, alpha=0.3, which='both', linestyle='--')

# Set nice axis limits
ax.set_xlim(k_true.min() * 0.9, k_true.max() * 1.1)

plt.tight_layout()
plt.show()

print(f"\n✓ Energy spectrum plot complete")
print(f"  This plot shows spectral bias: deviation from ground truth at high wavenumbers")

## Cell 8: Spectral Bias Quantification

Compute spectral bias metrics to quantify how well each model captures high-frequency content.

In [ ]:
from src.core.visualization.spectral_analysis import compute_spectral_bias_metric

print("="*70)
print("SPECTRAL BIAS METRICS")
print("="*70)
print("\nQuantifies how well each model captures different frequency ranges.")
print("Spectral Bias Ratio = High Freq Error / Low Freq Error")
print("  - Ratio > 2.0: Significant spectral bias (struggles with high frequencies)")
print("  - Ratio > 1.5: Moderate spectral bias")
print("  - Ratio ≤ 1.5: Low spectral bias (captures frequencies well)")
print("="*70)

# Compute metrics for each trained model
spectral_metrics = {}

for loss_type in ['baseline', 'bsp', 'sa-bsp']:
    key = f"{MODEL_ARCH}_{loss_type}"
    model_trained = trained_models[key]
    model_trained.eval()
    model_trained.to(device)
    
    with torch.no_grad():
        pred = model_trained(val_input)
    
    metrics = compute_spectral_bias_metric(pred.cpu(), val_target.cpu(), n_bins=32)
    spectral_metrics[loss_type] = metrics
    
    print(f"\n{MODEL_ARCH.upper()} + {loss_type.upper()} Loss:")
    print(f"  Low frequency error:   {metrics['low_freq_error']:.6f}")
    print(f"  Mid frequency error:   {metrics['mid_freq_error']:.6f}")
    print(f"  High frequency error:  {metrics['high_freq_error']:.6f}")
    print(f"  Spectral bias ratio:   {metrics['spectral_bias_ratio']:.4f}")
    
    # Interpretation
    if metrics['spectral_bias_ratio'] > 2.0:
        print(f"  → ⚠️  SIGNIFICANT spectral bias detected!")
        print(f"     Model struggles with high-frequency content")
    elif metrics['spectral_bias_ratio'] > 1.5:
        print(f"  → ⚡ MODERATE spectral bias")
        print(f"     Some difficulty with high frequencies")
    else:
        print(f"  → ✅ LOW spectral bias")
        print(f"     Model captures frequency content well")

# Create comparison visualization
print(f"\n{'='*70}")
print("Spectral Bias Comparison")
print(f"{'='*70}")

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))

# Bar plot 1: Frequency errors
x = np.arange(3)
width = 0.25

loss_types = ['baseline', 'bsp', 'sa-bsp']
low_errors = [spectral_metrics[lt]['low_freq_error'] for lt in loss_types]
mid_errors = [spectral_metrics[lt]['mid_freq_error'] for lt in loss_types]
high_errors = [spectral_metrics[lt]['high_freq_error'] for lt in loss_types]

ax1.bar(x - width, low_errors, width, label='Low Freq', color='#2ca02c', alpha=0.8)
ax1.bar(x, mid_errors, width, label='Mid Freq', color='#ff7f0e', alpha=0.8)
ax1.bar(x + width, high_errors, width, label='High Freq', color='#d62728', alpha=0.8)

ax1.set_xlabel('Loss Type', fontsize=12, fontweight='bold')
ax1.set_ylabel('Frequency Error', fontsize=12, fontweight='bold')
ax1.set_title('Frequency Range Errors', fontsize=14, fontweight='bold')
ax1.set_xticks(x)
ax1.set_xticklabels([lt.upper() for lt in loss_types])
ax1.legend(fontsize=11)
ax1.grid(True, alpha=0.3, axis='y')

# Bar plot 2: Spectral bias ratio
bias_ratios = [spectral_metrics[lt]['spectral_bias_ratio'] for lt in loss_types]
colors_bars = [colors_plot[lt] for lt in loss_types]

bars = ax2.bar(x, bias_ratios, color=colors_bars, alpha=0.8, edgecolor='black', linewidth=1.5)

# Add threshold lines
ax2.axhline(y=2.0, color='red', linestyle='--', linewidth=2, alpha=0.7, label='Significant bias threshold')
ax2.axhline(y=1.5, color='orange', linestyle='--', linewidth=2, alpha=0.7, label='Moderate bias threshold')

ax2.set_xlabel('Loss Type', fontsize=12, fontweight='bold')
ax2.set_ylabel('Spectral Bias Ratio', fontsize=12, fontweight='bold')
ax2.set_title('Spectral Bias Ratio (High/Low)', fontsize=14, fontweight='bold')
ax2.set_xticks(x)
ax2.set_xticklabels([lt.upper() for lt in loss_types])
ax2.legend(fontsize=10, loc='upper right')
ax2.grid(True, alpha=0.3, axis='y')

# Add value labels on bars
for i, (bar, ratio) in enumerate(zip(bars, bias_ratios)):
    height = bar.get_height()
    ax2.text(bar.get_x() + bar.get_width()/2., height + 0.05,
            f'{ratio:.2f}', ha='center', va='bottom', fontsize=11, fontweight='bold')

plt.suptitle(f'{MODEL_ARCH.upper()}: Spectral Bias Analysis', fontsize=16, fontweight='bold')
plt.tight_layout()
plt.show()

print(f"\n{'='*70}")
print("✅ Spectral bias analysis complete!")
print(f"{'='*70}")

## Summary

This notebook demonstrated:
1. ✓ Loading real CDON data with proper normalization
2. ✓ Creating neural operator models (DeepONet, FNO, UNet)
3. ✓ **Sequential training with all 3 loss functions** (Baseline, BSP, SA-BSP)
4. ✓ **Multi-loss comparison plots** showing training metrics
5. ✓ **Energy spectrum visualization** (E(k) vs wavenumber) to identify spectral bias
6. ✓ **Spectral bias quantification** with metrics and comparison plots

**Key Results:**
- All 3 loss types trained on the same model architecture
- Direct comparison shows which loss function best mitigates spectral bias
- Energy spectrum plot reveals how well each model captures high-frequency content
- Quantitative metrics identify spectral bias ratio for each approach

**Experiment with different configurations:**
- **Cell 3**: Change `MODEL_ARCH` to try different models ('deeponet', 'fno', 'unet')
- **Cell 5**: Adjust hyperparameters (epochs, learning rate, etc.) in TrainingConfig
- Run all cells sequentially to train and compare all loss types automatically!